In [116]:
import os
import csv
import json
import random
import jsonlines
import collections
import numpy as np

from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

# Process all schemas

In [47]:
schema_descr_path = '../data/schemas/descrs'
cgw_schema_rel_docs_path = '../data/cgw/schema_related/pos'
cgw_preds_args_path = '../data/cgw/preds_args'

data_split = {y: 'train' for y in range(1994, 2007)}
data_split.update({2007: 'dev', 2008: 'dev', 2009: 'test', 2010: 'test'})

for filename in os.listdir(schema_descr_path):
    if not filename.endswith('.json'):
        continue
    schema_name = filename[:-5]

    subset_docs_ids = set()

    # Collect schema-related docs' ids
    with jsonlines.open(f'{cgw_schema_rel_docs_path}/{schema_name}.jsonl') as reader:
        for doc in reader:
            subset_docs_ids.add(doc['id'])
    
    # Load schema events
    with open(f'{schema_descr_path}/{schema_name}.json') as fin:
        schema_descr = json.load(fin)
        filtered_events = schema_descr['predpatt']
    
    # Load all docs
    subset_raw_docs_ids = []
    subset_raw_docs = []

    for pred_arg_path in tqdm(glob(f'{cgw_preds_args_path}/*.jsonl')):
        with jsonlines.open(pred_arg_path) as reader:
            for doc in reader:
                assert doc['filename'].endswith('.comm')
                doc_id = doc['filename'][:-5]
                doc_split = data_split[int(doc_id[8:12])]
                if doc_split != 'train':
                    continue
                # if doc_id in subset_docs_ids:  # read only a subset of documents
                # pred_only_doc = [f'{a0 if ix == 0 else a1}_{ix}' for a0, a1, ix in doc['preds_args']]
                doc_repr = [(a0 if ix == 0 else a1) for a0, a1, ix in doc['preds_args']]
                doc_repr = [e for e in doc_repr if e in filtered_events]
                if len(doc_repr) > 5:
                    subset_raw_docs_ids.append(doc_id)
                    subset_raw_docs.append(doc_repr)
    
    subset_raw_docs_ids = np.array(subset_raw_docs_ids)
    assert len(subset_raw_docs) == len(subset_raw_docs_ids)

    # Vectorize docs
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
    subset_tfidf = vectorizer.fit_transform(subset_raw_docs)
    subset_tfidf = subset_tfidf.toarray()
    
    # Output to CSV
    header = [v for k, v in sorted([(v, k) for k, v in vectorizer.vocabulary_.items()])]
    vocab_size = len(vectorizer.vocabulary_)

    with open(f'{cgw_schema_rel_docs_path}/{schema_name}_binary_{vocab_size}.csv', 'w') as fout:
        writer = csv.writer(fout)
        writer.writerow(header)
        for row in np.int32(subset_tfidf > 0):
            writer.writerow(list(row))
    print(f'Output {len(subset_tfidf)} documents')

  0%|          | 0/197 [00:00<?, ?it/s]

Output 4237 documents


  0%|          | 0/197 [00:00<?, ?it/s]

Output 10582 documents


  0%|          | 0/197 [00:00<?, ?it/s]

Output 2984 documents


100%|██████████| 197/197 [01:46<00:00,  1.85it/s]


Output 5579 documents


100%|██████████| 197/197 [01:45<00:00,  1.86it/s]

Output 4472 documents


## Experiment with argument generalization

In [107]:
import nltk
from nltk.corpus import wordnet as wn

In [129]:
synsets = {
    "ce_001_protest": {
        "police": wn.synset('law_enforcement_agency.n.01').name(),
        "bush": wn.synset('politician.n.02').name(),
        "putin": wn.synset('politician.n.02').name(),
        "hussein": wn.synset('politician.n.02').name(),
        "hizballah": wn.synset('politician.n.02').name(),
        "hezbollah": wn.synset('politician.n.02').name(),
        "gore": wn.synset('politician.n.02').name(),
        "politician": wn.synset('politician.n.02').name(),
        "gore": wn.synset('politician.n.02').name(),
        
        "city": wn.synset('urban_area.n.01').name(),
        "broadway": wn.synset('urban_area.n.01').name(),
        "orleans": wn.synset('urban_area.n.01').name(),
        "tokyo": wn.synset('urban_area.n.01').name(),
        "selma": wn.synset('urban_area.n.01').name(),
        "guadalajara": wn.synset('urban_area.n.01').name(),
        "paris": wn.synset('urban_area.n.01').name(),
        "zaragoza": wn.synset('urban_area.n.01').name(),
        "beijing": wn.synset('urban_area.n.01').name(),
        "washington": wn.synset('urban_area.n.01').name(),
        "zaragoza": wn.synset('urban_area.n.01').name(),
        
        "demonstrator": wn.synset('reformer.n.01').name(),
        "protester": wn.synset('reformer.n.01').name(),
        "demonstration": wn.synset('protest.n.02').name(),
        "protest": wn.synset('protest.n.02').name(),
        "sit-in": wn.synset('protest.n.02').name(),
        "march": wn.synset('protest.n.02').name(),
        "rally": wn.synset('protest.n.02').name(),
        
        "woman": wn.synset('person.n.01').name(),
        "man": wn.synset('person.n.01').name(),
        "daughter": wn.synset('person.n.01').name(),
        "student": wn.synset('person.n.01').name(),
        "group": wn.synset('group.n.01').name(),

        "japan": wn.synset('political_unit.n.01').name(),
        "china": wn.synset('political_unit.n.01').name(),
        "israel": wn.synset('political_unit.n.01').name(),
        "isreal": wn.synset('political_unit.n.01').name(),
        "egypt": wn.synset('political_unit.n.01').name(),
        "lithuania": wn.synset('political_unit.n.01').name(),
        "lebanon": wn.synset('political_unit.n.01').name(),
        "haiti": wn.synset('political_unit.n.01').name(),
        "fiji": wn.synset('political_unit.n.01').name(),
        "nigeria": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
        "iraq": wn.synset('political_unit.n.01').name(),
        "uganda": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
    },
    "ce_005_disease_outbreak": {
        "organism": wn.synset('organism.n.01').name(),
        "fly": wn.synset('organism.n.01').name(),
        "cat": wn.synset('organism.n.01').name(),
        "rat": wn.synset('organism.n.01').name(),
        "mosquito": wn.synset('organism.n.01').name(),
        "cattle": wn.synset('organism.n.01').name(),
        "bird": wn.synset('organism.n.01').name(),
        "mouse": wn.synset('organism.n.01').name(),
        "gorilla": wn.synset('organism.n.01').name(),
        "deer": wn.synset('organism.n.01').name(),
        "horse": wn.synset('organism.n.01').name(),
        "worm": wn.synset('organism.n.01').name(),
        "bat": wn.synset('organism.n.01').name(),
        "sheep": wn.synset('organism.n.01').name(),
        "cow": wn.synset('organism.n.01').name(),
        "dog": wn.synset('organism.n.01').name(),
        "livestock": wn.synset('organism.n.01').name(),
        "bison": wn.synset('organism.n.01').name(),
        "duck": wn.synset('organism.n.01').name(),
        "frog": wn.synset('organism.n.01').name(),
        "chimpanzee": wn.synset('organism.n.01').name(),
        "pigeon": wn.synset('organism.n.01').name(),
        "swan": wn.synset('organism.n.01').name(),
        "shellfish": wn.synset('organism.n.01').name(),
        "pig": wn.synset('organism.n.01').name(),
        "salmon": wn.synset('organism.n.01').name(),
        "trout": wn.synset('organism.n.01').name(),
        "tick": wn.synset('organism.n.01').name(),
        "fish": wn.synset('organism.n.01').name(),
        "ferret": wn.synset('organism.n.01').name(),
        "rodent": wn.synset('organism.n.01').name(),
        "gerbil": wn.synset('organism.n.01').name(),
        "hamster": wn.synset('organism.n.01').name(),
        "raccoon": wn.synset('organism.n.01').name(),
        "lion": wn.synset('organism.n.01').name(),
        "crab": wn.synset('organism.n.01').name(),
        "elk": wn.synset('organism.n.01').name(),
        "mammal": wn.synset('organism.n.01').name(),
        "toad": wn.synset('organism.n.01').name(),
        "salmon": wn.synset('organism.n.01').name(),
        
        "poultry": wn.synset('food.n.02').name(),
        "beef": wn.synset('food.n.02').name(),
                
        "mumps": wn.synset('illness.n.01').name(),
        "meningitis": wn.synset('illness.n.01').name(),
        "polio": wn.synset('illness.n.01').name(),
        "tuberculosis": wn.synset('illness.n.01').name(),
        "dysentery": wn.synset('illness.n.01').name(),
        "leprosy": wn.synset('illness.n.01').name(),
        "cholera": wn.synset('illness.n.01').name(),
        "herpes": wn.synset('illness.n.01').name(),
        "fibrosis": wn.synset('illness.n.01').name(),
        "osteoporosis": wn.synset('illness.n.01').name(),
        "sclerosis": wn.synset('illness.n.01').name(),
        "influenza": wn.synset('illness.n.01').name(),
        "flu": wn.synset('illness.n.01').name(),
        "sickness": wn.synset('illness.n.01').name(),
        "infection": wn.synset('illness.n.01').name(),
        "illness": wn.synset('illness.n.01').name(),
        "injury": wn.synset('illness.n.01').name(),
        "leukemia": wn.synset('illness.n.01').name(),
        "carcinoma": wn.synset('illness.n.01').name(),
        "hiv": wn.synset('illness.n.01').name(),
        "asbestosis": wn.synset('illness.n.01').name(),
        "smallpox": wn.synset('illness.n.01').name(),
        "rabies": wn.synset('illness.n.01').name(),
        "anemia": wn.synset('illness.n.01').name(),
        "encephalitis": wn.synset('illness.n.01').name(),
        "zoonosis": wn.synset('illness.n.01').name(),
        "rinderpest": wn.synset('illness.n.01').name(),
        "bse": wn.synset('illness.n.01').name(),
        "osteoarthritis": wn.synset('illness.n.01').name(),
        "sepsis": wn.synset('illness.n.01').name(),
        "cancer": wn.synset('illness.n.01').name(),
        "trachoma": wn.synset('illness.n.01').name(),
        "malaria": wn.synset('illness.n.01').name(),
        "melanoma": wn.synset('illness.n.01').name(),
        "angioma": wn.synset('illness.n.01').name(),
        "plague": wn.synset('illness.n.01').name(),
        "sars": wn.synset('illness.n.01').name(),
        "pneumonia": wn.synset('illness.n.01').name(),
        "parkinson": wn.synset('illness.n.01').name(),
        "psoriasis": wn.synset('illness.n.01').name(),
        "arthritis": wn.synset('illness.n.01').name(),
        "mucopolysaccharidosis": wn.synset('illness.n.01').name(),
        "gonorrhea": wn.synset('illness.n.01').name(),
        "plague": wn.synset('illness.n.01').name(),
        "plague": wn.synset('illness.n.01').name(),
        "plague": wn.synset('illness.n.01').name(),
        "plague": wn.synset('illness.n.01').name(),
   
        "intestine": wn.synset('body_part.n.01').name(),
        "liver": wn.synset('body_part.n.01').name(),
        "lung": wn.synset('body_part.n.01').name(),
        "heart": wn.synset('body_part.n.01').name(),
        "lung": wn.synset('body_part.n.01').name(),
        "lung": wn.synset('body_part.n.01').name(),
        "lung": wn.synset('body_part.n.01').name(),
        "lung": wn.synset('body_part.n.01').name(),

        "bacillus": wn.synset('infectious_agent.n.01').name(),
        "spirochete": wn.synset('infectious_agent.n.01').name(),
        "microbe": wn.synset('infectious_agent.n.01').name(),
        "bacteria": wn.synset('infectious_agent.n.01').name(),
        "bacterium": wn.synset('infectious_agent.n.01').name(),
        "bacillus": wn.synset('infectious_agent.n.01').name(),
                
    },
    "election": {
        "candidate": wn.synset('campaigner.n.01').name(),
        "nominee": wn.synset('campaigner.n.01').name(),
        "candidate": wn.synset('campaigner.n.01').name(),
        "pac": wn.synset('campaigner.n.01').name(),
        
        "amendment": wn.synset('legislation.n.01').name(),
        "bill": wn.synset('legislation.n.01').name(),
        "treaty": wn.synset('legislation.n.01').name(),
        "contract": wn.synset('legislation.n.01').name(),
        "document.n.01": wn.synset('legislation.n.01').name(),
        
        "kennedy": wn.synset('civil_authority.n.01').name(),
        "administrative_unit.n.01": wn.synset('civil_authority.n.01').name(),
        "presiding_officer.n.01": wn.synset('civil_authority.n.01').name(),
        "clergyman.n.01": wn.synset('civil_authority.n.01').name(),
        "chief": wn.synset('civil_authority.n.01').name(),
        "chieftain": wn.synset('civil_authority.n.01').name(),
        "lawmaker": wn.synset('civil_authority.n.01').name(),
        "politician": wn.synset('civil_authority.n.01').name(),
        "bush": wn.synset('civil_authority.n.01').name(),
        "gore": wn.synset('civil_authority.n.01').name(),
        "clinton": wn.synset('civil_authority.n.01').name(),
        "nixon": wn.synset('civil_authority.n.01').name(),
        "president": wn.synset('civil_authority.n.01').name(),
        "governor": wn.synset('civil_authority.n.01').name(),
        "whip": wn.synset('civil_authority.n.01').name(),
        "senator": wn.synset('civil_authority.n.01').name(),
        "congresswoman": wn.synset('civil_authority.n.01').name(),
        "congressman": wn.synset('civil_authority.n.01').name(),
        "sheriff": wn.synset('civil_authority.n.01').name(),
        "deputy": wn.synset('civil_authority.n.01').name(),
        "representative": wn.synset('civil_authority.n.01').name(),
        "king": wn.synset('civil_authority.n.01').name(),
        "emperor": wn.synset('civil_authority.n.01').name(),
        "czar": wn.synset('civil_authority.n.01').name(),
        "judge": wn.synset('civil_authority.n.01').name(),
        "congress": wn.synset('civil_authority.n.01').name(),
        "parliament": wn.synset('civil_authority.n.01').name(),
        "senate": wn.synset('civil_authority.n.01').name(),
        "duma": wn.synset('civil_authority.n.01').name(),
        "eisenhower": wn.synset('civil_authority.n.01').name(),
        "house": wn.synset('civil_authority.n.01').name(),
        "jefferson": wn.synset('civil_authority.n.01').name(),
        "cabinet": wn.synset('civil_authority.n.01').name(),
        "official": wn.synset('civil_authority.n.01').name(),
        "representative.n.01": wn.synset('civil_authority.n.01').name(),
        "speaker": wn.synset('civil_authority.n.01').name(),
        "general": wn.synset('civil_authority.n.01').name(),
        "polity.n.02": wn.synset('civil_authority.n.01').name(),
        "legislator": wn.synset('civil_authority.n.01').name(),
        "hussein": wn.synset('civil_authority.n.01').name(),
        "mandela": wn.synset('civil_authority.n.01').name(),
        "reagan": wn.synset('civil_authority.n.01').name(),
        "buchanan": wn.synset('civil_authority.n.01').name(),
        "hussein": wn.synset('civil_authority.n.01').name(),
        "lawman.n.01": wn.synset('civil_authority.n.01').name(),
        
        "conservative": wn.synset('political_orientation.n.01').name(),
        "liberal": wn.synset('political_orientation.n.01').name(),
        "moderate": wn.synset('political_orientation.n.01').name(),
        "nationalist": wn.synset('political_orientation.n.01').name(),
        "center": wn.synset('political_orientation.n.01').name(),
        "middle": wn.synset('political_orientation.n.01').name(),
        "left": wn.synset('political_orientation.n.01').name(),
        "right": wn.synset('political_orientation.n.01').name(),
        "separatist": wn.synset('political_orientation.n.01').name(),
        "libertarian": wn.synset('political_orientation.n.01').name(),
        "democrat": wn.synset('political_orientation.n.01').name(),  # or party.n.01?
        "republican": wn.synset('political_orientation.n.01').name(),
        "communist": wn.synset('political_orientation.n.01').name(),
        "socialist": wn.synset('political_orientation.n.01').name(),
        "independent": wn.synset('political_orientation.n.01').name(),

        "people": wn.synset('people.n.01').name(),
        "adult.n.01": wn.synset('people.n.01').name(),
        "person.n.01": wn.synset('people.n.01').name(),
        
        "decision": wn.synset('vote.n.01').name(),
        "vote": wn.synset('vote.n.01').name(),
        "voting": wn.synset('vote.n.01').name(),
        
        "voter": wn.synset('voter.n.01').name(),
        "elector": wn.synset('voter.n.01').name(),
        
        "negative.n.01": wn.synset('result.n.03').name(),
        "affirmative.n.01": wn.synset('result.n.03').name(),
        "ending.n.04": wn.synset('result.n.03').name(),
        
        "colombia": wn.synset('political_unit.n.01').name(),
        "mexico": wn.synset('political_unit.n.01').name(),
        "georgia": wn.synset('political_unit.n.01').name(),
        "florida": wn.synset('political_unit.n.01').name(),
        "carolina": wn.synset('political_unit.n.01').name(),
        "colorado": wn.synset('political_unit.n.01').name(),
        "washington": wn.synset('political_unit.n.01').name(),
        "texas": wn.synset('political_unit.n.01').name(),
        "new_jersey.n.01": wn.synset('political_unit.n.01').name(),
        "missouri": wn.synset('political_unit.n.01').name(),
        "mississippi": wn.synset('political_unit.n.01').name(),
        "united_kingdom.n.01": wn.synset('political_unit.n.01').name(),
        "israel.n.01": wn.synset('political_unit.n.01').name(),
        "california": wn.synset('political_unit.n.01').name(),
        "peru": wn.synset('political_unit.n.01').name(),
        "province": wn.synset('political_unit.n.01').name(),
        "state": wn.synset('political_unit.n.01').name(),
        "country": wn.synset('political_unit.n.01').name(),
        "district": wn.synset('political_unit.n.01').name(),
        "municipality.n.01": wn.synset('political_unit.n.01').name(),
        "america": wn.synset('political_unit.n.01').name(),
        "ireland.n.01": wn.synset('political_unit.n.01').name(),
        
        "white": wn.synset('demographic.n.01').name(),
        "black": wn.synset('demographic.n.01').name(),
        "women": wn.synset('demographic.n.01').name(),
        "men": wn.synset('demographic.n.01').name(),
        "worker": wn.synset('demographic.n.01').name(),
        "gay": wn.synset('demographic.n.01').name(),
        "homosexual": wn.synset('demographic.n.01').name(),
        "jew": wn.synset('demographic.n.01').name(),
        "age": wn.synset('demographic.n.01').name(),
        "old": wn.synset('demographic.n.01').name(),
        "young": wn.synset('demographic.n.01').name(),
        "muslim.n.01": wn.synset('demographic.n.01').name(),
        "social_group.n.01": wn.synset('demographic.n.01').name(),
        "arab.n.01": wn.synset('demographic.n.01').name(),
        "south_american.n.01": wn.synset('demographic.n.01').name(),
        "american.n.01": wn.synset('demographic.n.01').name(),
        "asian.n.01": wn.synset('demographic.n.01').name(),
        "latino": wn.synset('demographic.n.01').name(),
        "inhabitant.n.01": wn.synset('demographic.n.01').name(),
        "absentee": wn.synset('demographic.n.01').name(),
        "central_american.n.01": wn.synset('demographic.n.01').name(),
        "catholic.n.01": wn.synset('demographic.n.01').name(),
        "christian.n.01": wn.synset('demographic.n.01').name(),
        "religious_person.n.01": wn.synset('demographic.n.01').name(),
        "juvenile.n.01": wn.synset('demographic.n.01').name(),
        
        "large_integer.n.01": wn.synset("number.n.01").name(),
        "proportion.n.01": wn.synset("number.n.01").name(),
        "common_fraction.n.01": wn.synset("number.n.01").name(),
        "digit.n.01": wn.synset("number.n.01").name(),
        "large_indefinite_quantity.n.01": wn.synset("number.n.01").name(),
    },
    "ce_040_arrest": {
        "murder": wn.synset('crime.n.01').name(),
        "smuggling": wn.synset('crime.n.01').name(),
        "intimidation": wn.synset('crime.n.01').name(),
        "assault": wn.synset('crime.n.01').name(),
        "scam": wn.synset('crime.n.01').name(),
        "kidnapping": wn.synset('crime.n.01').name(),
        "killing": wn.synset('crime.n.01').name(),
        
        "trafficker": wn.synset('criminal.n.01').name(),
        "dealer": wn.synset('criminal.n.01').name(),
        "shoplifter": wn.synset('criminal.n.01').name(),
        "rioter": wn.synset('criminal.n.01').name(),
        "troublemaker": wn.synset('criminal.n.01').name(),
        "plotter": wn.synset('criminal.n.01').name(),
        "abductor": wn.synset('criminal.n.01').name(),
        "offender": wn.synset('criminal.n.01').name(),
        "hijacker": wn.synset('criminal.n.01').name(),
        "rapist": wn.synset('criminal.n.01').name(),
        "stoner": wn.synset('criminal.n.01').name(),
        "predator": wn.synset('criminal.n.01').name(),
        "felon": wn.synset('criminal.n.01').name(),
        "criminal": wn.synset('criminal.n.01').name(),
        "pimp": wn.synset('criminal.n.01').name(),
        "abuser": wn.synset('criminal.n.01').name(),
        "attacker": wn.synset('criminal.n.01').name(),
        "perpetrator": wn.synset('criminal.n.01').name(),
        "pedophile": wn.synset('criminal.n.01').name(),
        "flasher": wn.synset('criminal.n.01').name(),
        
        # Group special witnesses together (doctor, psychologist, etc)

        "police": wn.synset('law_enforcement_agency.n.01').name(),
        "bush": wn.synset('politician.n.02').name(),

        "syria": wn.synset('political_unit.n.01').name(),
        "iraq": wn.synset('political_unit.n.01').name(),
        "yugoslavia": wn.synset('political_unit.n.01').name(),
        "zagreb": wn.synset('political_unit.n.01').name(),
        "nato": wn.synset('political_unit.n.01').name(),
        "europe": wn.synset('political_unit.n.01').name(),
        "eritrea": wn.synset('political_unit.n.01').name(),
        "lebanon": wn.synset('political_unit.n.01').name(),
        "haiti": wn.synset('political_unit.n.01').name(),
        "fiji": wn.synset('political_unit.n.01').name(),
        "nigeria": wn.synset('political_unit.n.01').name(),
        "cuba": wn.synset('political_unit.n.01').name(),
        "uganda": wn.synset('political_unit.n.01').name(),
        "pakistan": wn.synset('political_unit.n.01').name(),
        "iran": wn.synset('political_unit.n.01').name(),
        "belgium": wn.synset('political_unit.n.01').name(),
        "spain": wn.synset('political_unit.n.01').name(),
        "bengal": wn.synset('political_unit.n.01').name(),
        
        "diplomat": wn.synset('politician.n.02').name(),
        "vargas": wn.synset('politician.n.02').name(),
        "putin": wn.synset('politician.n.02').name(),
        "chiluba": wn.synset('politician.n.02').name(),
        "president": wn.synset('politician.n.02').name(),
    },
    "plane_crash": {
        "plane": wn.synset('airplane.n.01').name(),
        "helicopter": wn.synset('airplane.n.01').name(),
        "sailplane": wn.synset('airplane.n.01').name(),
        "airplane": wn.synset('airplane.n.01').name(),
        "glider": wn.synset('airplane.n.01').name(),
        "warplane": wn.synset('airplane.n.01').name(),
        "jetliner": wn.synset('airplane.n.01').name(),
        "aircraft": wn.synset('airplane.n.01').name(),
        "turboprop": wn.synset('airplane.n.01').name(),
        "warplane": wn.synset('airplane.n.01').name(),
        "warplane": wn.synset('airplane.n.01').name(),
        "warplane": wn.synset('airplane.n.01').name(),
    }
}

stopwords = {"we", "they", "who", "which", "that", "i", "it", "he", "she", "them", "him", "her", "u", "me", "a"}

In [130]:
schema_descr_path = '../data/schemas/descrs'
cgw_schema_rel_docs_path = '../data/cgw/schema_related/pos'
cgw_preds_args_path = '../data/cgw/preds_args'

data_split = {y: 'train' for y in range(1994, 2007)}
data_split.update({2007: 'dev', 2008: 'dev', 2009: 'test', 2010: 'test'})

In [140]:
schema_name = 'election'

# Load schema events
with open(f'{schema_descr_path}/{schema_name}.json') as fin:
    schema_descr = json.load(fin)
    filtered_events = {tuple(x) for x in schema_descr['predpatt_wordnet']}
    filtered_events_preds = {pred for pred, *_ in filtered_events}

In [142]:
# Load all docs
subset_raw_docs_ids = []
subset_raw_docs = []
subset_event_cntr = collections.Counter()

for pred_arg_path in tqdm(glob(f'{cgw_preds_args_path}/*.jsonl')):
    with jsonlines.open(pred_arg_path) as reader:
        for doc in reader:
            assert doc['filename'].endswith('.comm')
            doc_id = doc['filename'][:-5]
            doc_split = data_split[int(doc_id[8:12])]
            if doc_split != 'train':
                continue
            # if doc_id in subset_docs_ids:
            doc_repr = []
            for pred_args in doc['preds_args']:
                pred_idx = pred_args[2]
                arg_idx = 1 - pred_idx
                pred, arg = pred_args[pred_idx], pred_args[arg_idx].lower()
                if pred not in filtered_events_preds:
                    continue
                arg = lemmatizer.lemmatize(arg, pos=wn.NOUN)
                if arg.lower() in stopwords:
                    continue

                # Check if word is in the custom synset list
                # Use WordNet lookup if not
                if arg in synsets[schema_name]:
                    syn_root = synsets[schema_name][arg]
                else:
                    if not wn.synsets(arg, pos=wn.NOUN):
                        continue
                    syn = wn.synsets(arg, pos=wn.NOUN)[0]
                    if syn.hypernyms():
                        syn_root = syn.hypernyms()[0].name()
                    else:
                        syn_root = syn.name()
                if (pred, syn_root) in filtered_events:
                    # doc_repr.append(f'{pred}_{syn_root}_{pred_idx}')
                    doc_repr.append(f'{pred}__{syn_root}')
                elif (pred,) in filtered_events:
                    doc_repr.append(pred)
            if len(doc_repr) > 1:
                subset_raw_docs_ids.append(doc_id)
                subset_raw_docs.append(doc_repr)
                subset_event_cntr.update(doc_repr)

100%|██████████| 197/197 [02:06<00:00,  1.55it/s]


In [143]:
len(subset_raw_docs)

8179

In [144]:
subset_event_cntr.most_common()

[('outnumber', 6166),
 ('concede', 4016),
 ('vote__civil_authority.n.01', 2760),
 ('elect__civil_authority.n.01', 2099),
 ('oust', 1752),
 ('congratulate', 614),
 ('endorse__campaigner.n.01', 257),
 ('defeat__campaigner.n.01', 96),
 ('nominate__campaigner.n.01', 72),
 ('campaign__campaigner.n.01', 30)]

In [145]:
subset_raw_docs_ids = np.array(subset_raw_docs_ids)
assert len(subset_raw_docs) == len(subset_raw_docs_ids)

# Vectorize docs
vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
subset_tfidf = vectorizer.fit_transform(subset_raw_docs)
subset_tfidf = subset_tfidf.toarray()

# Output to CSV
header = [v for k, v in sorted([(v, k) for k, v in vectorizer.vocabulary_.items()])]
vocab_size = len(vectorizer.vocabulary_)

with open(f'{cgw_schema_rel_docs_path}/{schema_name}_binary_{vocab_size}_with_args.csv', 'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(header)
    for row in np.int32(subset_tfidf > 0):
        writer.writerow(list(row))

In [146]:
vocab_size

10

# Process narrative chains

In [ ]:
filtered_events = ['convict_0', 'face_1', 'sentence_0', 'arrest_0', 'accuse_1',
                   'plead_1', 'acquit_0', 'indict_0', 'testify_1', 'charge_0']

filtered_events_set = set(filtered_events)

In [ ]:
subset_raw_docs_ids = []
subset_raw_docs = []

with open('../data/cgw/nytimes_chains.txt') as reader:
    for ix, chain in tqdm(enumerate(reader)):
        pred_only_chain = [x.replace('->nsubj', '_1').replace('->dobj', '_0') for x in chain.strip().split()]
        subset_raw_docs_ids.append(ix)
        subset_raw_docs.append(pred_only_chain)

In [ ]:
subset_raw_docs_ids = np.array(subset_raw_docs_ids)

In [ ]:
%%time

vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False, vocabulary=filtered_events)
subset_tfidf = vectorizer.fit_transform(subset_raw_docs)
subset_tfidf = subset_tfidf.toarray()

vocab_relevant_mask = (subset_tfidf.sum(axis=1) > 0)
subset_tfidf = subset_tfidf[vocab_relevant_mask]
subset_tfidf_ids = subset_raw_docs_ids[vocab_relevant_mask]

In [ ]:
# Output to CSV

header = [v for k, v in sorted([(v, k) for k, v in vectorizer.vocabulary_.items()])]
vocab_size = len(vectorizer.vocabulary_)

with open(f'../data/cgw/schema_related/pos/{schema_name}_tfidf_cj_{vocab_size}.csv', 'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(header)
    for doc_id, row in zip(subset_tfidf_ids, subset_tfidf):
        writer.writerow(list(row))

---